# Calendar Sync Configuration

This notebook provides an easy way to set up and configure your calendar sync. It will help you:

1. Connect external calendars (Outlook, iCloud, etc.) to your Google Calendar
2. Set up sync frequencies and time windows
3. Generate a configuration that can run continuously
4. Install the sync as a background service that runs automatically when you log in

## Setup

First, make sure you've installed the required packages and set up your Google Calendar API credentials. If you haven't done this yet, please refer to the README.md file for detailed instructions.

> **Important**: Scroll to the bottom of this notebook to find the **Background Service** section, where you can install the calendar sync as a system service that runs automatically when you log in. This is the recommended way to keep your calendars in sync without keeping this notebook open.

## Calendar Configuration

Use the form below to configure your calendar settings. You can add as many calendars as you need.

In [ ]:
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import json
import os

# Default settings
default_days_back = 30
default_days_forward = 60
default_sync_interval = 5

# Try to load existing configuration
config_file = 'calendar_config.json'
if os.path.exists(config_file):
    with open(config_file, 'r') as f:
        try:
            saved_calendars = json.load(f).get('calendars', [])
        except:
            saved_calendars = []
else:
    saved_calendars = []

# Pre-fill with examples if no configuration exists
if not saved_calendars:
    saved_calendars = [
        { 
            'url': 'https://example.com/your-outlook-calendar.ics',
            'calendarName': 'Work Calendar',
            'daysBack': 30,
            'daysForward': 60,
            'syncInterval': 5
        },
        { 
            'url': 'https://example.com/your-personal-calendar.ics',
            'calendarName': 'Personal Calendar',
            'daysBack': 30,
            'daysForward': 60,
            'syncInterval': 5
        }
    ]

# Main calendar container
calendar_widgets = []

def create_calendar_form(url='', name='', days_back=default_days_back, days_forward=default_days_forward, sync_interval=default_sync_interval):
    """Create widgets for a single calendar configuration"""
    url_widget = widgets.Text(value=url, description='iCal URL:', style={'description_width': '120px'}, layout=widgets.Layout(width='90%'))
    name_widget = widgets.Text(value=name, description='Calendar Name:', style={'description_width': '120px'}, layout=widgets.Layout(width='50%'))
    days_back_widget = widgets.IntSlider(value=days_back, min=7, max=365, step=7, description='Days Back:', style={'description_width': '120px'})
    days_forward_widget = widgets.IntSlider(value=days_forward, min=7, max=365, step=7, description='Days Forward:', style={'description_width': '120px'})
    sync_interval_widget = widgets.IntSlider(value=sync_interval, min=1, max=60, step=1, description='Sync Interval (min):', style={'description_width': '120px'})
    
    remove_btn = widgets.Button(description='Remove', button_style='danger', layout=widgets.Layout(width='100px'))
    
    container = widgets.VBox([
        widgets.HBox([name_widget, remove_btn]),
        url_widget,
        widgets.HBox([days_back_widget, days_forward_widget]),
        sync_interval_widget,
        widgets.HTML(value='<hr style="height:2px;border-width:0;background-color:#ccc">')
    ])
    
    # Store all widgets together
    calendar_data = {
        'container': container,
        'url': url_widget,
        'name': name_widget,
        'days_back': days_back_widget,
        'days_forward': days_forward_widget,
        'sync_interval': sync_interval_widget,
        'remove_btn': remove_btn
    }
    
    calendar_widgets.append(calendar_data)
    
    # Handle removal
    def on_remove_clicked(b):
        calendar_widgets.remove(calendar_data)
        update_display()
    
    remove_btn.on_click(on_remove_clicked)
    
    return container

# Create main container and actions
main_container = widgets.VBox([])
add_btn = widgets.Button(description='Add Calendar', button_style='success')
save_btn = widgets.Button(description='Save Configuration', button_style='primary')
status_text = widgets.HTML(value='')

def update_display():
    """Update the display with all calendar widgets"""
    calendars = [widget['container'] for widget in calendar_widgets]
    main_container.children = calendars + [widgets.HBox([add_btn, save_btn]), status_text]

def on_add_clicked(b):
    """Handle adding a new calendar"""
    create_calendar_form()
    update_display()

def on_save_clicked(b):
    """Save the configuration to a file"""
    calendars_config = []
    
    for widget in calendar_widgets:
        calendars_config.append({
            'url': widget['url'].value,
            'calendarName': widget['name'].value,
            'daysBack': widget['days_back'].value,
            'daysForward': widget['days_forward'].value, 
            'syncInterval': widget['sync_interval'].value
        })
    
    # Save to file
    with open(config_file, 'w') as f:
        json.dump({"calendars": calendars_config}, f, indent=2)
    
    status_text.value = f"<div style=\"color: green; font-weight: bold;\">Configuration saved ({len(calendars_config)} calendars configured)</div>"
    
    # Get current username for the plist path
    username = os.environ.get('USER') or 'user'
    plist_path = os.path.expanduser(f"~/Library/LaunchAgents/com.{username}.calendarsync.plist")
    
    # Check if service is installed
    if os.path.exists(plist_path):
        status_text.value += "<div style=\"color: orange;\">Note: You may need to restart the background service for changes to take effect.</div>"

# Set up event handlers
add_btn.on_click(on_add_clicked)
save_btn.on_click(on_save_clicked)

# Add existing calendars
for calendar in saved_calendars:
    create_calendar_form(
        url=calendar.get('url', ''),
        name=calendar.get('calendarName', ''),
        days_back=calendar.get('daysBack', default_days_back),
        days_forward=calendar.get('daysForward', default_days_forward),
        sync_interval=calendar.get('syncInterval', default_sync_interval)
    )

# Display the form
update_display()
display(main_container)

## Run the Sync Process

Once you've configured your calendars, you can use the button below to start the synchronization process. The sync will run in the background and continue until you stop it.

In [ ]:
import sys
import subprocess
import threading
import time
import os
from IPython.display import clear_output

sync_process = None
stop_log_thread = False
log_thread = None
background_service_installed = False

def start_sync():
    global sync_process, stop_log_thread, log_thread
    
    if sync_process is not None:
        status_output.value = "<div style='color: orange; font-weight: bold;'>Sync is already running</div>"
        return
    
    # Start the sync process
    sync_process = subprocess.Popen([sys.executable, 'main.py'], 
                                   stdout=subprocess.PIPE, 
                                   stderr=subprocess.STDOUT,
                                   universal_newlines=True,
                                   bufsize=1)
    
    # Update UI
    start_btn.disabled = True
    stop_btn.disabled = False
    status_output.value = "<div style='color: green; font-weight: bold;'>Sync started</div>"
    log_output.value = "Starting calendar sync...\n"
    
    # Start log monitoring thread
    stop_log_thread = False
    log_thread = threading.Thread(target=monitor_logs)
    log_thread.daemon = True
    log_thread.start()

def stop_sync():
    global sync_process, stop_log_thread
    
    if sync_process is None:
        status_output.value = "<div style='color: orange; font-weight: bold;'>No sync process is running</div>"
        return
    
    # Stop the process
    sync_process.terminate()
    sync_process = None
    stop_log_thread = True
    
    # Update UI
    start_btn.disabled = False
    stop_btn.disabled = True
    status_output.value = "<div style='color: red; font-weight: bold;'>Sync stopped</div>"
    log_output.value += "\nSync process stopped.\n"

def install_background_service():
    global background_service_installed
    
    # Check if we're running on macOS or Linux
    if not os.path.exists("./install_background_service.sh"):
        service_status.value = "<div style='color: red;'>Error: install_background_service.sh not found</div>"
        return
    
    # Make the script executable
    subprocess.run(["chmod", "+x", "./install_background_service.sh"], check=True)
    
    # Run the installer
    result = subprocess.run(["./install_background_service.sh"],
                          stdout=subprocess.PIPE,
                          stderr=subprocess.STDOUT,
                          universal_newlines=True)
    
    if result.returncode == 0:
        background_service_installed = True
        service_status.value = f"<div style='color: green; font-weight: bold;'>Background service installed successfully!</div>"
        service_log.value = result.stdout
    else:
        service_status.value = "<div style='color: red; font-weight: bold;'>Failed to install background service</div>"
        service_log.value = result.stdout
    
    # Update buttons
    install_service_btn.disabled = background_service_installed
    uninstall_service_btn.disabled = not background_service_installed

def uninstall_background_service():
    global background_service_installed
    
    platform = subprocess.run(["uname", "-s"], stdout=subprocess.PIPE, text=True).stdout.strip()
    username = os.environ.get('USER', 'user')
    service_name = f"com.{username}.calendarsync"
    
    if platform == "Darwin":  # macOS
        # Unload the launchd service
        plist_path = os.path.expanduser(f"~/Library/LaunchAgents/{service_name}.plist")
        if os.path.exists(plist_path):
            result = subprocess.run(["launchctl", "unload", "-w", plist_path],
                                  stdout=subprocess.PIPE,
                                  stderr=subprocess.STDOUT,
                                  universal_newlines=True)
            
            # Remove the plist file
            subprocess.run(["rm", plist_path], check=False)
            
            service_log.value = "Service unloaded and removed from LaunchAgents."
            service_status.value = "<div style='color: blue; font-weight: bold;'>Background service uninstalled</div>"
            background_service_installed = False
        else:
            service_status.value = "<div style='color: orange;'>Service not found in LaunchAgents</div>"
    
    elif platform == "Linux":
        # Stop and disable the systemd service
        result = subprocess.run(["systemctl", "--user", "disable", "--now", "calendar-sync.service"],
                              stdout=subprocess.PIPE,
                              stderr=subprocess.STDOUT,
                              universal_newlines=True)
        
        # Remove the service file
        service_path = os.path.expanduser("~/.config/systemd/user/calendar-sync.service")
        if os.path.exists(service_path):
            subprocess.run(["rm", service_path], check=False)
        
        service_log.value = "Service stopped, disabled and removed from systemd."
        service_status.value = "<div style='color: blue; font-weight: bold;'>Background service uninstalled</div>"
        background_service_installed = False
    else:
        service_status.value = "<div style='color: orange;'>Unsupported operating system</div>"
    
    # Update buttons
    install_service_btn.disabled = background_service_installed
    uninstall_service_btn.disabled = not background_service_installed

def check_service_status():
    """Check if the background service is installed and running"""
    global background_service_installed
    
    platform = subprocess.run(["uname", "-s"], stdout=subprocess.PIPE, text=True).stdout.strip()
    username = os.environ.get('USER', 'user')
    service_name = f"com.{username}.calendarsync"
    
    if platform == "Darwin":  # macOS
        # Check launchd
        plist_path = os.path.expanduser(f"~/Library/LaunchAgents/{service_name}.plist")
        background_service_installed = os.path.exists(plist_path)
        
        if background_service_installed:
            # Check if it's actually loaded
            result = subprocess.run(["launchctl", "list", service_name],
                                  stdout=subprocess.PIPE,
                                  stderr=subprocess.PIPE,
                                  universal_newlines=True)
            
            if result.returncode == 0:
                service_status.value = "<div style='color: green; font-weight: bold;'>Background service is installed and running</div>"
            else:
                service_status.value = "<div style='color: orange; font-weight: bold;'>Background service is installed but not running</div>"
        else:
            service_status.value = "<div style='color: gray;'>Background service is not installed</div>"
    
    elif platform == "Linux":
        # Check systemd
        service_path = os.path.expanduser("~/.config/systemd/user/calendar-sync.service")
        background_service_installed = os.path.exists(service_path)
        
        if background_service_installed:
            # Check if it's running
            result = subprocess.run(["systemctl", "--user", "is-active", "calendar-sync.service"],
                                  stdout=subprocess.PIPE,
                                  stderr=subprocess.PIPE,
                                  universal_newlines=True)
            
            if "active" in result.stdout:
                service_status.value = "<div style='color: green; font-weight: bold;'>Background service is installed and running</div>"
            else:
                service_status.value = "<div style='color: orange; font-weight: bold;'>Background service is installed but not running</div>"
        else:
            service_status.value = "<div style='color: gray;'>Background service is not installed</div>"
    else:
        service_status.value = "<div style='color: orange;'>Unsupported operating system</div>"
    
    # Update buttons
    install_service_btn.disabled = background_service_installed
    uninstall_service_btn.disabled = not background_service_installed

def monitor_logs():
    """Read process output and update the log display"""
    global sync_process, stop_log_thread
    
    while not stop_log_thread and sync_process is not None:
        try:
            # Read up to 1000 chars at a time from the process output
            output = sync_process.stdout.readline()
            if output:
                # Update the log display
                log_output.value += output
                
                # Keep only the last 20 lines to avoid excessive memory usage
                lines = log_output.value.split('\n')
                if len(lines) > 20:
                    log_output.value = '\n'.join(lines[-20:])
            
            # Check if process has terminated
            if sync_process.poll() is not None:
                log_output.value += "\nSync process terminated.\n"
                sync_process = None
                start_btn.disabled = False
                stop_btn.disabled = True
                status_output.value = "<div style='color: orange; font-weight: bold;'>Sync process terminated</div>"
                break
            
            time.sleep(0.1)
            
        except Exception as e:
            log_output.value += f"\nError monitoring logs: {str(e)}\n"
            break

# Create UI components for sync
start_btn = widgets.Button(description="Start Sync", button_style="success")
stop_btn = widgets.Button(description="Stop Sync", button_style="danger")
stop_btn.disabled = True

status_output = widgets.HTML(value="<div style='color: gray;'>Sync not running</div>")
log_output = widgets.Textarea(value="", layout=widgets.Layout(width='100%', height='200px'))

# Create UI components for background service
service_heading = widgets.HTML(value="<h3>Background Service</h3><p>Install the calendar sync as a system service that runs automatically when you log in.</p>")
install_service_btn = widgets.Button(description="Install Background Service", button_style="primary")
uninstall_service_btn = widgets.Button(description="Uninstall Background Service", button_style="warning")
service_status = widgets.HTML(value="<div style='color: gray;'>Checking service status...</div>")
service_log = widgets.Textarea(value="", layout=widgets.Layout(width='100%', height='150px'))

# Check initial service status
check_service_status()

# Set up event handlers
start_btn.on_click(lambda b: start_sync())
stop_btn.on_click(lambda b: stop_sync())
install_service_btn.on_click(lambda b: install_background_service())
uninstall_service_btn.on_click(lambda b: uninstall_background_service())

# Display the UI
display(widgets.HTML("<h3>Manual Sync</h3><p>Run the sync tool directly from the notebook (will stop when notebook is closed).</p>"))
display(widgets.HBox([start_btn, stop_btn]))
display(status_output)
display(widgets.Label("Log output:"))
display(log_output)

# Display the background service UI
display(service_heading)
display(widgets.HBox([install_service_btn, uninstall_service_btn]))
display(service_status)
display(widgets.Label("Service installation log:"))
display(service_log)